In [7]:
import numpy as np
from ase import Atoms
from ase.io import write
import subprocess

# Create a UCl3 molecule using ASE
symbols = ["U", "Cl", "Cl", "Cl"]
positions = np.array([
    [0.000, 0.000, 0.000],     # U
    [0.000, 0.000, 2.5],       # Cl
    [2.2, 0.0, -1.25],         # Cl
    [-2.2, 0.0, -1.25]         # Cl
])
ucl3 = Atoms(symbols=symbols, positions=positions)

# Save to file as xyz
write("ucl3.xyz", ucl3)

# Write Packmol input file
n_molecules = 1
box_size = 10
with open("packmol.inp", "w") as f:
    f.write(f"""tolerance 2.0
filetype xyz
output ucl3_random.xyz

structure ucl3.xyz
  number {n_molecules}
  inside box 0. 0. 0. {box_size} {box_size} {box_size}
end structure
""")

# Run packmol
result = subprocess.run("packmol < packmol.inp", shell=True)
print(result.stdout)
if result.returncode != 0:
    print("Packmol failed:")
    print(result.stderr)



################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                                                             Version 21.0.1 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: http://m3g.iqm.unicamp.br/packmol 

  Reading input file... (Control-C aborts)
  Types of coordinate files specified: xyz
  Seed for random number generator:      1234567
  Output file: ucl3_random.xyz
  Reading coordinate file: ucl3.xyz
  Number of independent structures:            1
  The structures are: 
  Structure            1 :Properties=species:S:1:pos:R:3 pbc="F F F"(           4  atoms)
  Maximum number of GENCAN loops for all molecule packing:          200
  Distance tolerance:    2.0000000000000000     
  Number of molecules of type          

In [8]:
def xyz_to_cp2k_coord(xyz_file, coord_file="coord.inp"):
    with open(xyz_file, "r") as f:
        lines = f.readlines()[2:]  # skip first two lines of .xyz format

    with open(coord_file, "w") as f:
        f.write("&COORD\n")
        for line in lines:
            tokens = line.split()
            element = tokens[0]
            x, y, z = map(float, tokens[1:4])
            f.write(f"  {element:<2}  {x:.6f}  {y:.6f}  {z:.6f}\n")
        f.write("&END COORD\n")

# Run this
xyz_to_cp2k_coord("ucl3_random.xyz")
